#### Importando bibliotecas

In [5]:
%run funcoes_automatizadas.py

🔄 Testando router0 → 192.168.2.2
🔄 Testando router2 → 192.168.0.2
🔄 Testando router3 → 192.168.1.2


In [2]:
#Função que mostra a tabela de roteamento
def run_ospf_ip_route():  
  return subprocess.check_output([
      "docker", "exec", "router0", "vtysh", "-c", "show ip route"
  ]).decode()

In [3]:
run_ospf_ip_route()

'Codes: K - kernel route, C - connected, S - static, R - RIP,\n       O - OSPF, I - IS-IS, B - BGP, E - EIGRP, N - NHRP,\n       T - Table, v - VNC, V - VNC-Direct, A - Babel, F - PBR,\n       f - OpenFabric,\n       > - selected route, * - FIB route, q - queued, r - rejected, b - backup\n       t - trapped, o - offload failure\n\nK>* 0.0.0.0/0 [0/0] via 192.168.0.254, eth0, 03:38:20\nO   192.168.0.0/24 [110/10] is directly connected, eth0, weight 1, 03:38:19\nC>* 192.168.0.0/24 is directly connected, eth0, 03:38:20\nO>* 192.168.1.0/24 [110/20] via 192.168.0.1, eth0, weight 1, 03:37:37\nO>* 192.168.2.0/24 [110/20] via 192.168.0.1, eth0, weight 1, 03:37:37\n'

<!-- 
 -->

<!-- import subprocess
import time
import threading

# # ----- ROUTING TABLES -----
# def run_ospf_ip_route():  
#   return subprocess.check_output([
#       "docker", "exec", "router_ospf_0", "vtysh", "-c", "show ip route"
#   ]).decode()

# def run_rip_ip_route():
#   return subprocess.check_output([
#       "docker", "exec", "router_rip_0", "vtysh", "-c", "show ip route"
#   ]).decode()  

# ospf_ip_route = run_ospf_ip_route()
# rip_ip_route = run_rip_ip_route()

# # Get OSPF and RIP lines
# ospf_rows = ospf_ip_route.strip().splitlines()
# rip_rows = rip_ip_route.strip().splitlines()


# print("----------OSPF IP Route:-----------")
# for row in ospf_rows:
#   print(row)

# print("\n----------RIP IP Route:-----------")
# for row in rip_rows:
#   print(row)
# # -------------------------------------

# ----- SHUTDOWN CONTAINER AFTER A PING -----
def ping_loop(target_ip, origin_container, results, stop_event):
  while not stop_event.is_set():
    try:
      output = subprocess.check_output([
        'docker', 'exec', origin_container, 'ping', '-c', '1', '-W', '1', target_ip
      ], stderr=subprocess.DEVNULL)
      print(output.decode())
      success = True
    except subprocess.CalledProcessError:
      success = False
    timestamp = time.time()
    results.append((timestamp, success))
    time.sleep(0.5)

def measure_with_docker(routing_procol, target_ip, origin_container, container_to_stop, delay_before_stop=5):  
  print(f"🚀 Starting ping loop from {origin_container} to {target_ip}... Using {routing_procol}")
  results = []
  stop_event = threading.Event()
  thread = threading.Thread(target=ping_loop, args=(target_ip, origin_container, results, stop_event))
  thread.daemon = True
  thread.start()
  print(f"⏳ Waiting {delay_before_stop}s before shutting router down...")
  time.sleep(delay_before_stop)

  print(f"⛔ Stoping container {container_to_stop}...")
  subprocess.run(['docker', 'stop', container_to_stop], stdout=subprocess.DEVNULL)

  print("⏱️ Waiting ping to work again...")
  stop_at, back_at = None, None
  while True:
    for i in range(1, len(results)):
      previous, current = results[i-1], results[i]
      previous_success, current_success = previous[1], current[1]
      if not stop_at and previous_success and not current_success:
        stop_at = current[0] 
      elif stop_at and not previous_success and current_success:
        back_at = current[0]
        break
    if back_at:
      break
    time.sleep(0.1)
 
  stop_event.set()
  thread.join()
  print(f"🕒 {routing_procol} reconvergence time: {back_at - stop_at:.2f}s")

# ----- MEASURE OSPF -----
measure_with_docker(
  routing_procol="OSPF",
  target_ip="192.168.5.2", 
  origin_container="router_ospf_3",
  container_to_stop="router_ospf_0", 
  delay_before_stop=1
)
print("\n💤 Aguardando 5s antes de medir o RIP...\n")
time.sleep(5)
# # ----- MEASURE RIP -----
# measure_with_docker(
#   routing_procol="RIP",
#   target_ip="192.168.1.2",
#   origin_container="router_rip_3",
#   container_to_stop="router_rip_0",
#   delay_before_stop=1
# ) -->